In [2]:
pip install azure-cosmos

Note: you may need to restart the kernel to use updated packages.


In [2]:
COSMOS_ENDPOINT = 'https://azcdb-krm.documents.azure.com:443/'
COSMOS_KEY = 'QQmmlUlCYuzidQHUjVm0VFzQYkd0itAd1wmMiy5qVsahZGRDeFSxefMp06CzhUYIfGaDysReAWpSACDbedDEzw=='

DATABASE_NAME = 'Test'
CONTAINER_NAME = 'retail'

In [3]:
from azure.cosmos import CosmosClient, exceptions, PartitionKey

In [4]:
client = CosmosClient(COSMOS_ENDPOINT, COSMOS_KEY)

In [5]:
database = client.create_database_if_not_exists(id=DATABASE_NAME)

In [6]:
container = database.create_container_if_not_exists(
    id=CONTAINER_NAME,
    partition_key=PartitionKey(path="/id"),
    offer_throughput=400
)

In [8]:
documento = {
    "id": "1",
    "nombre": "Producto A",
    "descripcion": "Descripcion del producto A",
    "precio": 29.99,
    "categoria": "Electronica"
}
type(documento)

dict

In [10]:
try:
    container.create_item(body=documento)
    print("Documento insertado exitosamente.")
except exceptions.CosmosResourceExistsError:
    print("El documento ya existe.")

El documento ya existe.


In [14]:
query = 'SELECT * FROM c' 

items = list(container.query_items(
    query=query,
    enable_cross_partition_query=True
))

print(items)

[{'id': '1', 'nombre': 'Producto A', 'descripcion': 'Descripcion del producto A', 'precio': 24.99, 'categoria': 'Electronica', '_rid': 'Nb4eAO2pNK0BAAAAAAAAAA==', '_self': 'dbs/Nb4eAA==/colls/Nb4eAO2pNK0=/docs/Nb4eAO2pNK0BAAAAAAAAAA==/', '_etag': '"10009ceb-0000-0100-0000-670712fd0000"', '_attachments': 'attachments/', 'departamento': 'Artefactos', '_ts': 1728516861}]


In [17]:
try:
    item = container.read_item(item="2", partition_key="1")
    item['precio'] = 24.99
    item['departamento'] = 'Artefactos'
    container.replace_item(item=item, body=item)
except Exception as e:
    print("Error al actualizar el documento: ", e)

Error al actualizar el documento:  (NotFound) Entity with the specified id does not exist in the system. More info: https://aka.ms/cosmosdb-tsg-not-found, 
RequestStartTime: 2024-10-09T23:40:41.9251617Z, RequestEndTime: 2024-10-09T23:40:41.9272512Z,  Number of regions attempted:1
{"systemHistory":[{"dateUtc":"2024-10-09T23:39:43.0161342Z","cpu":0.232,"memory":418184872.000,"threadInfo":{"isThreadStarving":"False","threadWaitIntervalInMs":0.0501,"availableThreads":32765,"minThreads":64,"maxThreads":32767},"numberOfOpenTcpConnection":751},{"dateUtc":"2024-10-09T23:39:53.0256539Z","cpu":0.107,"memory":418205368.000,"threadInfo":{"isThreadStarving":"False","threadWaitIntervalInMs":0.0513,"availableThreads":32765,"minThreads":64,"maxThreads":32767},"numberOfOpenTcpConnection":758},{"dateUtc":"2024-10-09T23:40:03.0355905Z","cpu":0.107,"memory":418184024.000,"threadInfo":{"isThreadStarving":"False","threadWaitIntervalInMs":0.0558,"availableThreads":32764,"minThreads":64,"maxThreads":32767},"n

In [19]:
container.delete_item(item="2", partition_key='1')

CosmosResourceNotFoundError: (NotFound) Entity with the specified id does not exist in the system. More info: https://aka.ms/cosmosdb-tsg-not-found, 
RequestStartTime: 2024-10-09T23:45:59.7681848Z, RequestEndTime: 2024-10-09T23:45:59.7705497Z,  Number of regions attempted:1
{"systemHistory":[{"dateUtc":"2024-10-09T23:45:03.3331776Z","cpu":0.183,"memory":417973940.000,"threadInfo":{"isThreadStarving":"False","threadWaitIntervalInMs":0.0549,"availableThreads":32765,"minThreads":64,"maxThreads":32767},"numberOfOpenTcpConnection":734},{"dateUtc":"2024-10-09T23:45:13.3431354Z","cpu":0.509,"memory":417975420.000,"threadInfo":{"isThreadStarving":"False","threadWaitIntervalInMs":0.0264,"availableThreads":32765,"minThreads":64,"maxThreads":32767},"numberOfOpenTcpConnection":734},{"dateUtc":"2024-10-09T23:45:23.3530868Z","cpu":0.283,"memory":417993244.000,"threadInfo":{"isThreadStarving":"False","threadWaitIntervalInMs":0.0426,"availableThreads":32765,"minThreads":64,"maxThreads":32767},"numbe